In [1]:
from lammps.calculator.worker import LammpsWorker
import asyncio
import time
loop = asyncio.get_event_loop()

In [2]:
workers = LammpsWorker(cwd="lammps_workers", num_workers=1)
loop.run_until_complete(workers.create())

In [3]:
script = """
log  lammps.log
units  metal
dimension  3
boundary  p p p
atom_style  full
read_data  initial.data
kspace_style  pppm 0.000010
pair_style  buck/coul/long 10
pair_coeff 1 1 1309362.2766468062 0.104 0.0
pair_coeff 1 2 9892.357 0.20199 0.0
pair_coeff 2 2 2145.7345 0.3 30.2222
set type 2 charge -1.400000
set type 1 charge 1.400000
dump  1 all custom 1 mol.lammpstrj id type x y z fx fy fz
dump_modify  1 sort id
thermo_style  custom step etotal pxx pyy pzz pxy pxz pyz
run  0
"""

files = {
    'initial.data': b"""pymatgen autogenerated data file

8 atoms

2 atom types

0 4.1990858 xlo xhi
0 4.1990858 ylo yhi
0 4.1990858 zlo zhi
2.571198492157551e-16 2.571198492157551e-16 2.5711984921575517e-16 xy xz yz

Masses

1 24.305
2 15.9994

Atoms

1 1 1 1.4 0.0 0.0 0.0
2 1 1 1.4 3.3763224914577695e-16 2.0995429 2.0995429
3 1 1 1.4 2.0995429 0.0 2.0995429
4 1 1 1.4 2.0995429000000003 2.0995429 2.571198492157551e-16
5 1 2 -1.4 2.0995429000000003 2.0995429 2.0995429000000003
6 1 2 -1.4 2.0995429 0.0 1.2855992460787756e-16
7 1 2 -1.4 3.3763224914577695e-16 2.0995429 1.2855992460787756e-16
8 1 2 -1.4 0.0 0.0 2.0995429
"""
}

async def run_lammps_job():
    start_time = time.perf_counter()
    job = await workers.submit(script, files, properties={'stress', 'forces', 'energy'})
    await job.future
    print(2, job.stdout, time.perf_counter() - start_time)
    return job

job = loop.run_until_complete(run_lammps_job())

2 b"LAMMPS (14 May 2016)\nReading data file ...\n  triclinic box = (0 0 0) to (4.19909 4.19909 4.19909) with tilt (2.5712e-16 2.5712e-16 2.5712e-16)\n  1 by 1 by 1 MPI processor grid\n  reading atoms ...\n  8 atoms\nFinding 1-2 1-3 1-4 neighbors ...\n Special bond factors lj:   0          0          0         \n Special bond factors coul: 0          0          0         \n  0 = max # of 1-2 neighbors\n  0 = max # of 1-3 neighbors\n  0 = max # of 1-4 neighbors\n  1 = max # of special neighbors\nSetting atom values ...\n  4 settings made for charge\nSetting atom values ...\n  4 settings made for charge\nWARNING: No fixes defined, atoms won't move (../verlet.cpp:55)\nPPPM initialization ...\nWARNING: Using 12-bit tables for long-range coulomb (../kspace.cpp:316)\n  G vector (1/distance) = 0.356365\n  grid = 8 8 8\n  stencil order = 5\n  estimated absolute RMS force accuracy = 2.47121e-05\n  estimated relative force accuracy = 1.71616e-06\n  using double precision FFTs\n  3d grid and FFT v

In [ ]:
print(job.stdout.decode())

In [10]:
job.future.result

<function Future.result>